# Regression Project

Reviewing regression from Jamie Chan's Machine Learning with Python for Beginners

In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
df = pd.read_csv("./Data/reviews.csv")